In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import io
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.animation import FuncAnimation
import glob
import random

from scipy.optimize import curve_fit

from astropy.io import fits
from astropy.time import Time
from astropy import units as u
import scipy.integrate as integrate
import lightkurve

from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

import math
import datetime as dt
import lightkurve.io as l
from lightkurve.io.generic import read_generic_lightcurve

In [2]:
def clean_text(str):
    s = result = ''.join([i for i in archivo[20:] if not i.isdigit()])
    t = s.replace('_curves.txt', '')
    u = t.replace('+', '')
    u = u.replace('-', '')
    if 'KsJs' in u:
        u = u.replace('KsJs', 'Ks Js')
    if u[0] == '_':
        u = u[1:]
    if 'AB' in u:
        u = u.replace('AB', '')
    return u

In [3]:
# Function is created in order to eliminate characters which are not Printable ASCII characters that appear 
# from the data transfer from repository
def textcleaner(str1):
    w = str(str1).split('\n')[0]
    x = str(w)
    y = str(x.split('  ')[2])
    z = y.replace("'", "")
    return z

In [4]:
# Function is created in order to eliminate characters which are not Printable ASCII characters that appear 
# from the data transfer from repository
def numcleaner(str1):
    w = str(str1).split('\n')[0]
    x = str(w)
    x.split('  ')
    if ' NaN' in x:
        y = str(x.split('  ')[1])
        z = y.replace("'", "")
        a = z.replace(" ", "")
        return a
    else:
        w = str(str1).split('\n')[0]
        x = str(w)
        y = str(x.split('  ')[2])
        z = y.replace("'", "")
        return z

In [5]:
#repository like the one we have online
general_data = pd.read_csv('/Users/kmanj/FITS_BD_Files/Repository_Data/Repository.csv', encoding ='latin1')

In [6]:
dir = '/Users/kmanj/Manjavacas Research/Metchev_2015/Metchev2015/NV/'
dir_save = '/Users/kmanj/FITS_BD_Files/Data/Metchev_2015/NV/'

In [7]:
dwarfs_files = os.listdir(dir)
#seperating the ch1 and ch2
dwarf_dat_ch1 = [archive for archive in dwarfs_files if archive.endswith("1.txt")]
dwarf_dat_ch2 = [archive for archive in dwarfs_files if archive.endswith("ch2.txt")]
print(dwarf_dat_ch1)
print(dwarf_dat_ch2)

['SDSS1416_mainbinA0001quad01.txt', 'ULAS1416_mainbinA0002quad01.txt']
['SDSS1416_mainbinA0001quad01_ch2.txt', 'ULAS1416_mainbinA0002quad01_ch2.txt']


In [8]:
new_name_file = ['SDSS J141624.08+134826.7', 'ULAS J141623.94+134836.3']
new_name = ['SDSS J141624.08+134826.7', 'ULAS J141623.94+134836.3']

### Channel 1

In [22]:
dwarfs_data = []
for i in range(len(new_name_file)):
    data = pd.read_csv("{}{}".format(dir,dwarf_dat_ch1[i]), header=None, sep='\t')
    dwarfs_data.append(data)

In [23]:
dwarfs_data

[             0         1
 0     0.023493  1.000375
 1     0.060611  1.001801
 2     0.096985  0.999087
 3     0.133392  1.002278
 4     0.169778  0.999509
 ..         ...       ...
 373  13.647043  0.998742
 374  13.683428  1.002310
 375  13.719801  1.000809
 376  13.756186  0.999368
 377  13.801679  0.999784
 
 [378 rows x 2 columns],
              0         1
 0     0.024226  0.999687
 1     0.060611  0.987785
 2     0.096985  0.986320
 3     0.133392  1.004006
 4     0.169778  0.979768
 ..         ...       ...
 371  13.636122  1.003697
 372  13.672506  1.006388
 373  13.708891  0.992933
 374  13.745276  1.008518
 375  13.796221  1.007524
 
 [376 rows x 2 columns]]

In [24]:
Reference = 'Metchev et al 2015 ApJ 799 154'
Telescope = 'Spitzer Space Telescope'
Instrument = 'Infrared Array Camera'
Date_obs = 'Published January 27, 2015'
Band_obs = 'Ch1'

In [25]:
Object = ['SDSS J141624.08+134826.7', 'ULAS J141623.94+134836.3']
SpectralType = ['L5', 'T7,5']
Jmag = ['13.148 +/- 0.025','17.259 +/- 0.017']
Hmag = ['12.456 +/- 0.028','17.581 +/- 0.03']
Kmag = ['12.114 +/- 0.023','18.933 +/- 0.244']
RA = ['214.100355','214.100355']
Dec = ['13.807318','13.807318']

In [26]:
name_files = []
for i in range(len(dwarfs_data)):
    lc = l.LightCurve(time = list(dwarfs_data[i][0]), flux = list(dwarfs_data[i][1]) * u.dimensionless_unscaled)
    name = dir_save + Object[i]
    contador = 1
    while name + '_Ch1_Metc2015.fits' in name_files:
        name = name + '_' + str(contador)
        contador += 1
    else:
        lc.to_fits(path = name + '_Ch1_Metc2015.fits')
        name_files.append(name + '_Ch1_Metc2015.fits')

In [27]:
for i in range(len(dwarfs_data)):
    with fits.open(name_files[i], mode = 'update') as f:
        f[0].header['OBJECT'] = Object[i]
        f[0].header['SPECTYPE'] = SpectralType[i]
        f[0].header['TIMESYS'] = 'mjd'
        f[0].header['TELESCOP'] = Telescope
        f[0].header['INSTRUME'] = Instrument
        f[0].header['FILTER'] = Band_obs
        f[0].header['VOREF'] = Reference
        f[0].header['RA_OBJ'] = RA[i]
        f[0].header['DEC_OBJ'] = Dec[i]
        f[0].header['JMAG'] = Jmag[i] 
        f[0].header['HMAG'] = Hmag[i] 
        f[0].header['KMAG'] = Kmag[i] 
        f[1].header['TUNIT1'] = 'Hrs'

### Channel 2

In [16]:
dwarfs_data = []
for i in range(len(new_name_file)):
    data = pd.read_csv("{}{}".format(dir,dwarf_dat_ch2[i]), header=None, sep='\t')
    dwarfs_data.append(data)

In [17]:
dwarfs_data

[             0         1
 0    14.043651  1.000869
 1    14.083677  0.999091
 2    14.120085  0.998192
 3    14.156447  1.001238
 4    14.192843  1.000348
 ..         ...       ...
 183  20.727736  1.000468
 184  20.764109  0.999476
 185  20.800495  1.001321
 186  20.836868  1.001000
 187  20.884186  1.001098
 
 [188 rows x 2 columns],
              0         1
 0    14.043651  1.004354
 1    14.080044  1.009623
 2    14.116441  1.002803
 3    14.152815  0.992935
 4    14.189199  0.997778
 ..         ...       ...
 182  20.716803  0.999860
 183  20.753199  1.003913
 184  20.789585  1.009883
 185  20.825958  1.001554
 186  20.878721  0.997511
 
 [187 rows x 2 columns]]

In [18]:
Reference = 'Metchev et al 2015 ApJ 799 154'
Instrument = 'Infrared Array Camera'
Date_obs = 'Published January 27, 2015'
Band_obs = 'Ch2'

In [19]:
Object = ['SDSS J141624.08+134826.7', 'ULAS J141623.94+134836.3']
SpectralType = ['L5', 'T7,5']
Jmag = ['13.148 +/- 0.025','17.259 +/- 0.017']
Hmag = ['12.456 +/- 0.028','17.581 +/- 0.03']
Kmag = ['12.114 +/- 0.023','18.933 +/- 0.244']
RA = ['214.100355','214.100355']
Dec = ['13.807318','13.807318']

In [20]:
name_files = []
for i in range(len(dwarfs_data)):
    lc = l.LightCurve(time = list(dwarfs_data[i][0]), flux = list(dwarfs_data[i][1]) * u.dimensionless_unscaled)
    name = dir_save + Object[i]
    contador = 1
    while name + '_Ch2_Metc2015.fits' in name_files:
        name = name + '_' + str(contador)
        contador += 1
    else:
        lc.to_fits(path = name + '_Ch2_Metc2015.fits')
        name_files.append(name + '_Ch2_Metc2015.fits')

In [21]:
for i in range(len(dwarfs_data)):
    with fits.open(name_files[i], mode = 'update') as f:
        f[0].header['OBJECT'] = Object[i]
        f[0].header['SPECTYPE'] = SpectralType[i]
        f[0].header['TIMESYS'] = 'mjd'
        f[0].header['TELESCOP'] = Telescope
        f[0].header['INSTRUME'] = Instrument
        f[0].header['FILTER'] = Band_obs
        f[0].header['VOREF'] = Reference
        f[0].header['RA_OBJ'] = RA[i]
        f[0].header['DEC_OBJ'] = Dec[i]
        f[0].header['JMAG'] = Jmag[i] 
        f[0].header['HMAG'] = Hmag[i] 
        f[0].header['KMAG'] = Kmag[i] 
        f[1].header['TUNIT1'] = 'Hrs'